In [1]:
import pandas as pd
import numpy as np
import requests
import folium
import polyline
import urllib
import matplotlib.pyplot as plt
import seaborn as sns
import time
import math
import json
from branca.element import Figure
from sklearn.cluster import KMeans
from geopy import distance

In [2]:
# retrieve number of vehicles from previous notebook
%store -r available_vehicles 

In [3]:
available_vehicles

4

In [4]:
# set daily orders in ./data/order_for_today.csv and generate sample_daily_order list

order_for_today = pd.read_csv('../data/order_for_today.csv')
sample_daily_order = order_for_today['0'].tolist()

In [5]:
# api can only handle 100 locations

sample_daily_order = sample_daily_order[:80]
sample_daily_order.append('10 woodlands terrace, singapore')
len(sample_daily_order)

81

In [6]:
# import df_lat_lng for coordinates

df_lat_lng = pd.read_csv('../data/df_lat_lng.csv')

In [7]:
# import distance_matrix

distance_matrix = pd.read_csv('../data/distance_array.csv')
distance_matrix.set_index(keys = 'Unnamed: 0', inplace = True) # setting name as index

In [8]:
# create new df for sample daily order 

sample_daily_order_df = pd.DataFrame(index = sample_daily_order)

In [9]:
# concat sample_daily_order_df with distance_matrix

sample_daily_order_df = pd.concat([sample_daily_order_df, distance_matrix], axis=1).reindex(sample_daily_order_df.index)

In [10]:
sample_daily_order_ordered = []

In [11]:
for i in range(len(sample_daily_order)):
    try:
        if i == 0:
            sample_daily_order_ordered.append('10 woodlands terrace, singapore')
            sample_daily_order_df.drop(index = '10 woodlands terrace, singapore', inplace = True)
            next_location = sample_daily_order_df['10 woodlands terrace, singapore'].sort_values().index.tolist()[0]
        else:
            current_location = next_location
            sample_daily_order_ordered.append(current_location)
            sample_daily_order_df.drop(index = current_location, inplace = True)
            next_location = sample_daily_order_df[current_location].sort_values().index.tolist()[0]
    except:
        pass
        
        

In [12]:
sample_daily_order_ordered

['10 woodlands terrace, singapore',
 '334 sembawang close, singapore',
 '118a canberra cres, singapore',
 '848 yishun street 81, singapore',
 '632 ang mo kio ave 4, singapore',
 '631 ang mo kio avenue 4, singapore',
 '505 ang mo kio street 53, singapore',
 '530 ang mo kio avenue 10, singapore',
 '339 ang mo kio ave 1, singapore',
 '453 ang mo kio avenue 1, singapore',
 '284 bishan street 22, singapore',
 '511 bishan street 13, singapore',
 '504 bishan street 13, singapore',
 '210 toa payoh lorong 8, singapore',
 '22a lorong 7 toa payoh, singapore',
 '29 bendemeer road, singapore',
 '17 upper boon keng road, singapore',
 '116 aljunied, singapore',
 '119 aljunied, singapore',
 '89 circuit road, singapore',
 '121 paya lebar way, singapore',
 '3014 ubi road 1, singapore',
 '210 hougang street 21, singapore',
 'chomp chomp food centre, singapore',
 '4 cardiff grove, singapore',
 '631 hougang ave 8, singapore',
 '682 hougang ave 8, singapore',
 '923 hougang street 91, singapore',
 '2 jalan s

In [13]:
sample_daily_order_nested = []
location_per_vehicle = (len(sample_daily_order_ordered)- (len(sample_daily_order_ordered) % available_vehicles)) / available_vehicles

for vehicle in range(available_vehicles + 1):
    if vehicle == 0:
        start = int(vehicle)
        end = int(location_per_vehicle - 1)
        sample_daily_order_nested.append(sample_daily_order_ordered[start:end])
    else:
        start = int(vehicle * location_per_vehicle)
        end = int(((vehicle + 1) * location_per_vehicle) - 1)
        sample_daily_order_nested.append(sample_daily_order_ordered[start:end])

In [14]:
0 - 15
16 - 31
32 - 47
48 - 62
63 - 79
79 - 85

-6

In [15]:
sample_daily_order_nested

[['10 woodlands terrace, singapore',
  '334 sembawang close, singapore',
  '118a canberra cres, singapore',
  '848 yishun street 81, singapore',
  '632 ang mo kio ave 4, singapore',
  '631 ang mo kio avenue 4, singapore',
  '505 ang mo kio street 53, singapore',
  '530 ang mo kio avenue 10, singapore',
  '339 ang mo kio ave 1, singapore',
  '453 ang mo kio avenue 1, singapore',
  '284 bishan street 22, singapore',
  '511 bishan street 13, singapore',
  '504 bishan street 13, singapore',
  '210 toa payoh lorong 8, singapore',
  '22a lorong 7 toa payoh, singapore',
  '29 bendemeer road, singapore',
  '17 upper boon keng road, singapore',
  '116 aljunied, singapore',
  '119 aljunied, singapore'],
 ['121 paya lebar way, singapore',
  '3014 ubi road 1, singapore',
  '210 hougang street 21, singapore',
  'chomp chomp food centre, singapore',
  '4 cardiff grove, singapore',
  '631 hougang ave 8, singapore',
  '682 hougang ave 8, singapore',
  '923 hougang street 91, singapore',
  '2 jalan sel